In [35]:

from osdatahub import FeaturesAPI, Extent
import geopandas as gpd
import folium
import matplotlib.pyplot as plt
import mapclassify as mc
from shapely.geometry import Point
import os

key = os.environ.get('OS_API_KEY')
bbox = (-2.2361, 53.4746, -2.2234, 53.4805)
extent = Extent.from_bbox(bbox, "EPSG:4326")
crs = "EPSG:4326"

product = "Zoomstack_DistrictBuildings"
features = FeaturesAPI(key, product, extent)
results = features.query()
results_gdf = gpd.GeoDataFrame.from_features(results['features'], crs=crs)

d:\anaconda3\envs\masters\lib\site-packages\osdatahub\FeaturesAPI\features_api.py:100: UserWarning: The features API does not support EPSG:4326 and will return an empty features list.
  warnings.warn("The features API does not support EPSG:4326 and will return an empty features list.")


In [21]:
results_gdf.explore()

In [33]:
def OSparam_feature(u,v,rad,product,key,clip):
    extent = Extent.from_radius((u,v), rad, "EPSG:27700")
    features = FeaturesAPI(key, product, extent)
    results = features.query(limit=500)
    if len(results['features']) == 0:
        out = 0
    TA_gdf = gpd.GeoDataFrame.from_features(results['features'])

    if clip == True:
        patch = Point(u,v).buffer(rad)
        d = {'col1': ['name1'], 'geometry': [patch]}
        patch_df = gpd.GeoDataFrame(d, crs="EPSG:27700")
        TA_gdf = TA_gdf.clip(patch)
        # TA_gdf = TA_gdf[TA_gdf.geometry.intersects(Point(u,v).buffer(rad))]

    try:
        gd = (TA_gdf['Theme'] == 'Buildings') & (~np.isnan(TA_gdf['RelH2']))
        out = np.average(TA_gdf['RelH2'][gd], weights=TA_gdf['Shape_Area'][gd])
    except TypeError:
        out = 0
    return out, TA_gdf

    

In [37]:
X = 18700
Y = 44777
radius = 100
clip = True

extent = Extent.from_radius((X,Y), radius, "EPSG:27700")
features = FeaturesAPI(key, "topographic_area", extent)
results = features.query(limit=500)
print(results)


# area, TA = OSparam_feature(X, Y, radius, "topographic_area",key=key, clip=clip)

{"crs": "EPSG:27700", "features": [], "type": "FeatureCollection"}
